In [ ]:
# libraries
import numpy as np
import pandas as pd
import networkx as nx
import pickle as pk
from glob import glob
import os, json, random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime,date, timedelta
from matplotlib import pyplot as plt
import matplotlib
from datetime import date
import re,ast,random
import matplotlib.dates as mdates
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = 'Arial'
import seaborn as sns
from scipy.stats import mannwhitneyu
from collections import Counter 
import matplotlib
import calendar
import matplotlib.font_manager as font_manager

In [ ]:
# extract data:

new_sample =False
if new_sample:
    cleaned_data = []
    for i in range(3):
        init_data = pd.read_csv('cleaned_data_part-'+str(i)+'.csv',lineterminator='\n')
        init_data['date'] = [date.fromtimestamp(t) if str(t) !='nan' else np.nan for t in init_data['time_epoch'].values]
        init_data['tweetId'] = init_data['tweetId'].astype(str)
        init_data['conversation_id'] = init_data['conversation_id'].astype(str)
        init_data['twitterAuthorScreenname'] = init_data['twitterAuthorScreenname'].astype(str)
        init_data = init_data.drop_duplicates(subset='tweetId')
        cleaned_data.append(init_data)
    cleaned_data = pd.concat(cleaned_data)
    del init_data
    cleaned_data = cleaned_data.drop_duplicates(subset='tweetId')
    init_data0 = cleaned_data.loc[cleaned_data['date']<=datetime(2022,12,1).date(),].sample(frac=0.2)
    init_data1 = cleaned_data.loc[cleaned_data['date']>datetime(2022,12,1).date(),]    
    del cleaned_data
    cleaned_data = pd.concat([init_data0,init_data1])
    user_cleaned_data = cleaned_data.groupby('twitterAuthorScreenname')
    
    
    cleaned_data.to_csv('equal_sample.csv',index=False)
else:
    cleaned_data = pd.read_csv('equal_sample.csv',lineterminator='\n')#'sampled_data_3.csv',lineterminator='\n')
    cleaned_data['twitterAuthorScreenname'] = cleaned_data['twitterAuthorScreenname'].astype(str).values
    cleaned_data['date'] = [date.fromtimestamp(t) if str(t) !='nan' else np.nan for t in cleaned_data['time_epoch'].values]
    user_cleaned_data = cleaned_data.groupby('twitterAuthorScreenname')
    if False:
        coord_data_hashtag = []
        coord_data_rt = []
        coord_data_time = []    
        for i in range(3):
            part_data = pd.read_csv('cleaned_data_part-'+str(i)+'.csv',lineterminator='\n')
            part_data['twitterAuthorScreenname'] = part_data['twitterAuthorScreenname'].astype(str)
            part_users = set(part_data['twitterAuthorScreenname'].drop_duplicates().values.tolist())
            user_part_data = part_data.groupby('twitterAuthorScreenname')
            coord_data_hashtag.append(pd.concat([user_part_data.get_group(u) for u in hashtag_G.nodes() if u in part_users]))
            coord_data_rt.append(pd.concat([user_part_data.get_group(u) for u in retweet_G.nodes() if u in part_users]))
            coord_data_time.append(pd.concat([user_part_data.get_group(u) for u in time_G.nodes() if u in part_users]))
        coord_data_hashtag = pd.concat(coord_data_hashtag)
        coord_data_rt = pd.concat(coord_data_rt)
        coord_data_time = pd.concat(coord_data_time)
print(len(cleaned_data))
cleaned_data = cleaned_data.drop_duplicates(subset='tweetId')
print(len(cleaned_data))


In [ ]:
# most frequent cashtags

cashtag_data = cleaned_data.dropna(subset='cashtags')
cashtags = []
for c in cashtag_data['cashtags'].values:
    cashtags += [cc['tag'] for cc in ast.literal_eval(c)]

fig,ax = plt.subplots(1,1,figsize=(5,5))
plt.bar(['$'+key for key,value in Counter(cashtags).most_common(10)],[value for key,value in Counter(cashtags).most_common(10)],color='royalblue')
plt.ylabel('Frequency')
plt.xticks(rotation=60)
plt.tight_layout()
plt.savefig('CashtagFreq.pdf')
plt.show()

In [ ]:
# cashtags over time

matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['font.family'] = 'Arial'
plt.style.use('default')
dates = [[datetime(2022,1,1).date()+timedelta(days=7*i+j) for j in range(7)] for i in range(80)]
weeks = [d+timedelta(days=3) for d in np.array(dates)[:,0]]
weeks1=[w for w in weeks if w <= date(2022,12,1)]
weeks2=[w for w in weeks if w >= date(2023,1,1)]
tweet_types = ['retweet','reply','tweet','quote']

min_month,min_year,max_month,max_year = [1,2022,6,2023]
num_bins = int(((weeks2[-1]-datetime(min_year,min_month,1).date()).days)/7)
all_bins = [datetime(min_year,1,1)+timedelta(days=7*i) for i in range(num_bins)]
all_bins = pd.to_datetime(all_bins)
crypto = cleaned_data.dropna(subset=['cashtags'])[['conversation_id','created_at','date']]
events = pd.to_datetime(crypto['date'])
hist = np.histogram(events,all_bins)

events_all = pd.to_datetime(cleaned_data['date'])#[datetime.strptime(str(t).split('T')[0],'%Y-%m-%d').date() for t in cleaned_data['created_at'].values],)
hist_all = np.histogram(events_all,all_bins)
percent = hist[0]/hist_all[0]*100

fig, ax = plt.subplots(1,1,figsize=(6,6))    
plt.plot(hist[1][:-1][hist[1][:-1]<pd.to_datetime('2022-12-01')],hist[0][hist[1][:-1]<pd.to_datetime('2022-12-01')],'k-',linewidth=2)
plt.plot(hist[1][:-1][(hist[1][:-1]>pd.to_datetime('2022-12-31')) & (hist[0]>0)],hist[0][(hist[1][:-1]>pd.to_datetime('2022-12-31')) & (hist[0]>0)],'k-',linewidth=2)
plt.plot([hist[1][:-1][hist[1][:-1]<pd.to_datetime('2022-12-01')][-1],hist[1][:-1][hist[1][:-1]>pd.to_datetime('2022-12-31')][0]],[hist[0][hist[1][:-1]<pd.to_datetime('2022-12-01')][-1],hist[0][hist[1][:-1]>pd.to_datetime('2022-12-31')][0]],'k:',linewidth=1)
print([np.mean(hist[0][(hist[1][:-1]>pd.to_datetime('2022-04-01'))&(hist[1][:-1]<pd.to_datetime('2022-10-27')) & (hist[0]>0)]),np.mean(list(hist[0][(hist[1][:-1]<pd.to_datetime('2022-12-01')) & (hist[1][:-1]>pd.to_datetime('2022-10-27')) & (hist[0]>0)])+list(hist[0][(hist[1][:-1]>pd.to_datetime('2022-12-31')) & (hist[0]>0)]))])
print(mannwhitneyu(hist[0][(hist[1][:-1]>pd.to_datetime('2022-04-01'))&(hist[1][:-1]<pd.to_datetime('2022-10-27')) & (hist[0]>0)],list(hist[0][(hist[1][:-1]<pd.to_datetime('2022-12-01')) & (hist[1][:-1]>pd.to_datetime('2022-10-27')) & (hist[0]>0)])+list(hist[0][(hist[1][:-1]>pd.to_datetime('2022-12-31')) & (hist[0]>0)])))

plt.plot([date(2022,10,27)]*2,[0,1000],'r-')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.ylim([0,1000])
plt.xticks(rotation=60)  
plt.ylabel('Number cashtag tweets')
plt.savefig('NumberCashtag.pdf')

plt.show()


fig, ax = plt.subplots(1,1,figsize=(4,4))    
plt.plot(hist[1][:-1][hist[1][:-1]<pd.to_datetime('2022-12-01')],percent[hist[1][:-1]<pd.to_datetime('2022-12-01')],'k-',linewidth=2)
plt.plot(hist[1][:-1][(hist[1][:-1]>pd.to_datetime('2022-12-31')) & (hist[0]>0)],percent[(hist[1][:-1]>pd.to_datetime('2022-12-31')) & (percent>0)],'k-',linewidth=2)
plt.plot([hist[1][:-1][hist[1][:-1]<pd.to_datetime('2022-12-01')][-1],hist[1][:-1][hist[1][:-1]>pd.to_datetime('2022-12-31')][0]],[percent[hist[1][:-1]<pd.to_datetime('2022-12-01')][-1],percent[hist[1][:-1]>pd.to_datetime('2022-12-31')][0]],'k:',linewidth=1)
print([np.mean(percent[(hist[1][:-1]>pd.to_datetime('2022-04-01'))&(hist[1][:-1]<pd.to_datetime('2022-10-27'))]),np.mean(list(percent[(hist[1][:-1]<pd.to_datetime('2022-12-01')) & (hist[1][:-1]>pd.to_datetime('2022-10-27'))])+list(percent[hist[1][:-1]>pd.to_datetime('2022-12-31')]))])
print(mannwhitneyu(percent[(hist[1][:-1]>pd.to_datetime('2022-04-01'))&(hist[1][:-1]<pd.to_datetime('2022-10-27'))],list(percent[(hist[1][:-1]<pd.to_datetime('2022-12-01')) & (hist[1][:-1]>pd.to_datetime('2022-10-27'))])+list(percent[hist[1][:-1]>pd.to_datetime('2022-12-31')])))


max_percent = int(max(percent)+1)
plt.plot([date(2022,10,27)]*2,[0,max_percent],'r-')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.xticks(rotation=60)  
plt.ylim([0,1.5])#max_percent])
plt.ylabel('% Posts With Cashtags ($...)')
plt.tight_layout()
plt.savefig('PercentCashtag.pdf')
plt.show()



weeks = np.array(weeks)
conv_ids = crypto['conversation_id'].values
all_crypto_data = pd.merge(cleaned_data,crypto,on=['conversation_id','created_at','date'])
all_days = all_crypto_data['date'].drop_duplicates().sort_values().values
day_crypto = all_crypto_data.groupby('date')
fig, ax = plt.subplots(1,1,figsize=(6,6))    
mean_rts = []
for ii in range(len(dates)):
    rt_data = [day_crypto.get_group(d) for d in dates[ii] if d in all_days]
    ml=np.nan
    if len(rt_data)>0:
        ml=pd.concat(rt_data)['retweet_count'].dropna().values.mean() 
    mean_rts.append(ml)
mean_rts = np.array(mean_rts)
plt.plot(weeks[weeks<datetime(2022,12,1).date()],mean_rts[weeks<datetime(2022,12,1).date()],'b-',linewidth=2)
plt.plot(weeks[weeks>datetime(2022,12,31).date()],mean_rts[weeks>datetime(2022,12,31).date()],'b-',linewidth=2)
plt.plot([weeks[weeks<datetime(2022,12,1).date()][-1],weeks[weeks>datetime(2022,12,31).date()][0]],[mean_rts[weeks<datetime(2022,12,1).date()][-1],mean_rts[weeks>datetime(2022,12,31).date()][0]],'b:',linewidth=1)
plt.plot([date(2022,10,27)]*2,[0,20000],'r-')

ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.ylim([1,20000])
plt.xticks(rotation=60)  
plt.ylabel('Number reposts')
plt.yscale('log')
plt.savefig('NumberCashtagRTs.pdf')
plt.show()

fig, ax = plt.subplots(1,1,figsize=(6,6)) 
mean_likes = []
for ii in range(len(dates)):
    like_data = [day_crypto.get_group(d) for d in dates[ii] if d in all_days]
    ml=np.nan
    if len(like_data)>0:
        ml=pd.concat(like_data)['like_count'].dropna().values.mean() 
    mean_likes.append(ml)
mean_likes = np.array(mean_likes)
plt.plot(weeks[weeks<datetime(2022,12,1).date()],mean_likes[weeks<datetime(2022,12,1).date()],'r-',linewidth=2)
plt.plot(weeks[weeks>datetime(2022,12,31).date()],mean_likes[weeks>datetime(2022,12,31).date()],'r-',linewidth=2)
plt.plot([weeks[weeks<datetime(2022,12,1).date()][-1],weeks[weeks>datetime(2022,12,31).date()][0]],[mean_likes[weeks<datetime(2022,12,1).date()][-1],mean_likes[weeks>datetime(2022,12,31).date()][0]],'r:',linewidth=1)
#plt.plot(weeks,mean_likes,'k-',linewidth=2)
#print([np.mean(mean_likes[weeks<datetime(2022,10,27).date()]),np.mean(list(mean_likes[(weeks<datetime(2022,12,1).date()) & (weeks>datetime(2022,10,27).date())])+list(mean_likes[weeks>datetime(2022,12,31).date()]))])
#print(mannwhitneyu(mean_likes[weeks<datetime(2022,10,27).date()],list(mean_likes[(weeks<datetime(2022,12,1).date()) & (weeks>datetime(2022,10,27).date())])+list(mean_likes[weeks>datetime(2022,12,31).date()])))

plt.plot([date(2022,10,27)]*2,[0,1000],'r-')
#plt.fill_between([date(2022, 12, 1),date(2023, 1, 1)],[1000,1000],color='k',alpha=0.2)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.ylim([1,10])
plt.xticks(rotation=60)  .ylabel('Number likes')
plt.yscale('log')
plt.savefig('NumberCashtagLikes.pdf')
plt.show()

import numpy as np
import matplotlib.pyplot as plt
from math import pi

import pandas as pd
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


x1_raw=hist[1][:-1][hist[1][:-1]<pd.to_datetime('2022-10-27')]
x1 = pd.to_datetime(x1_raw).astype('int64').values
x1 = [[xx] for xx in x1]
y1=percent[hist[1][:-1]<pd.to_datetime('2022-10-27')]
from sklearn.linear_model import LinearRegression
reg1 = LinearRegression().fit(x1, y1)


x2_raw=hist[1][:-1][(hist[1][:-1]>pd.to_datetime('2022-10-27')) &(~np.isnan(percent)) & ((hist[1][:-1]<pd.to_datetime('2022-12-01'))  | (hist[1][:-1]>pd.to_datetime('2022-12-31')))]
x2 = pd.to_datetime(x2_raw).astype('int64').values
x2 = [[xx] for xx in x2]
y2=percent[(hist[1][:-1]>pd.to_datetime('2022-10-27'))&(~np.isnan(percent)) & ((hist[1][:-1]<pd.to_datetime('2022-12-01'))  | (hist[1][:-1]>pd.to_datetime('2022-12-31')))]

from sklearn.linear_model import LinearRegression
reg2 = LinearRegression().fit(x2, y2)


from scipy import stats
import numpy as np

slope, intercept, r_value, p_value, std_err = stats.linregress([xx[0]/10**18 for xx in x1],y1)

confidence_interval = 1.96*std_err

print([slope,intercept,p_value])

slope, intercept, r_value, p_value, std_err = stats.linregress([xx[0]/10**18 for xx in x2],y2)

confidence_interval = 1.96*std_err

print([slope,intercept,p_value])


max_percent=1.2
matplotlib.rcParams['font.size']=15
matplotlib.rcParams['font.family']='Arial'
fig, ax = plt.subplots(1,1,figsize=(4.8,5.12))
sns.regplot(x=[xx[0]/10**18 for xx in x1], y=y1,scatter_kws={'color':'b'},line_kws={'color':'r'},ax=ax)
sns.regplot(x=[xx[0]/10**18 for xx in x2], y=y2,scatter_kws={'color':'b'},line_kws={'color':'r'},ax=ax)

x = [datetime(2022,m,1,0,0).timestamp()/10**9 for m in range(1,13,2)] + [datetime(2023,m,1,0,0).timestamp()/10**9 for m in range(1,6,2)]
labels = [datetime(2022,m,1,0,0).strftime("%b %Y") for m in range(1,13,2)] + [datetime(2023,m,1,0,0).strftime("%b %Y") for m in range(1,6,2)]
plt.plot([datetime(2022,10,27).timestamp()/10**9]*2,[0,max_percent+0.3],'k-')
plt.xticks(x, labels, rotation=60)
plt.ylim([0,1.1])
plt.ylabel('% Posts With Cashtags ($...)')
plt.tight_layout()
plt.savefig('PercentCashtag.pdf',transparent=True)
plt.show()